# Data Preparation

In [43]:
import io 
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.metrics
import tensorflow as tf

from PIL import Image
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [44]:
#Loading the data (numpy arrays)

path_train = r'C:\Users\aldan\OneDrive\Escritorio\Study Notes\0 - Data Sets\Img_array_dataset\Primary categories - Train.npz'
path_val = r'C:\Users\aldan\OneDrive\Escritorio\Study Notes\0 - Data Sets\Img_array_dataset\Primary categories - Validation.npz'
path_test = r'C:\Users\aldan\OneDrive\Escritorio\Study Notes\0 - Data Sets\Img_array_dataset\Primary categories - Test.npz'

data_train = np.load(path_train)
data_val = np.load(path_val)
data_test = np.load(path_test)

In [45]:
#Displaying the data we loaded
display(
    dict(data_train).keys()
    , dict(data_val).keys()
    , dict(data_test).keys()
)

dict_keys(['images', 'labels'])

dict_keys(['images', 'labels'])

dict_keys(['images', 'labels'])

In [46]:
#Checking it's shape
data_train['images'].shape

(12963, 120, 90, 3)

In [47]:
#Checking unique values
np.unique(data_train['labels']), data_train['labels'].shape

(array([0, 1, 2]), (12963,))

In [48]:
#Dividing the data for train, validation, test 
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_val['images']
labels_val = data_val['labels']

images_test = data_test['images']
labels_test = data_test['labels']

In [49]:
#Scaling the values for optimization purposes
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

# Modeling

In [50]:
#Setting up the variables and hyperparameters
EPOCHS = 15
BATCH_SIZE = 64

HP_FILTER_SIZES = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_number', hp.Discrete([32,64,96,128]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(r'logs/Model_1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZES, HP_FILTER_NUM],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )

In [51]:
def train_test_model(hparams, session_num):

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters = hparams[HP_FILTER_NUM], kernel_size = hparams[HP_FILTER_SIZES], activation = 'relu', input_shape = (120, 90, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(filters = hparams[HP_FILTER_NUM], kernel_size = 3, activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy'])

    log_dir = r'logs/Model_1/fit/run-{}'.format(session_num)

    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure   
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')     
    
    def log_confusion_matrix(epoch, logs):
        """
        Saves the confusion matrix event 

        Args:
            epoch: number of epochs
            logs: directory to save the logs
        """
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Glasses/Sunglasses', 'Trousers/Jeans', 'Shoes'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,labels_val),
        verbose = 2
    )
     
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(images_val,labels_val)
    
    # Saving the current model for future reference
    model.save(r"saved_models\Model_1\Run-{}".format(session_num))
    
    return accuracy    

In [52]:
# Creating a function to train the model and log the results
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        # record the values used in this trial
        hp.hparams(hparams)  
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [53]:
# For loop to run and train the model using the different hyperparameters
session_num = 1

# Set the for nested loop for all the combinations between hyperparameters
for filter_size in HP_FILTER_SIZES.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:

        hparams = {
            HP_FILTER_SIZES: filter_size,
            HP_FILTER_NUM: filter_num
        }

        run_name = "run-%d" % session_num
        # print useful information
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        # train and export the model
        run('Logs/Model_1/hparam_tuning/' + run_name, hparams, session_num)

        session_num += 1

--- Starting trial: run-1
{'filter_size': 5, 'filter_number': 64}
Epoch 1/15
51/51 [==============================] - 4s 84ms/step
203/203 - 163s - loss: 0.0888 - accuracy: 0.9784 - val_loss: 0.0144 - val_accuracy: 0.9975 - 163s/epoch - 805ms/step
Epoch 2/15
51/51 [==============================] - 4s 69ms/step
203/203 - 148s - loss: 0.0185 - accuracy: 0.9965 - val_loss: 0.0155 - val_accuracy: 0.9981 - 148s/epoch - 730ms/step
Epoch 3/15
51/51 [==============================] - 3s 67ms/step
203/203 - 125s - loss: 0.0156 - accuracy: 0.9976 - val_loss: 0.0070 - val_accuracy: 0.9988 - 125s/epoch - 617ms/step
Epoch 4/15
51/51 [==============================] - 3s 67ms/step
203/203 - 125s - loss: 0.0078 - accuracy: 0.9991 - val_loss: 0.0055 - val_accuracy: 0.9988 - 125s/epoch - 618ms/step
Epoch 5/15
51/51 [==============================] - 4s 69ms/step
203/203 - 126s - loss: 0.0048 - accuracy: 0.9995 - val_loss: 0.0052 - val_accuracy: 0.9988 - 126s/epoch - 619ms/step
Epoch 6/15
51/51 [======

INFO:tensorflow:Assets written to: saved_models\model_2\Run-1\assets


INFO:tensorflow:Assets written to: saved_models\model_2\Run-1\assets
